imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import torch.nn.functional as F 

import numpy as np
import math
import scipy
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import skimage as ski
import time
%matplotlib widget

functions

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def hist_hue(pixel_data_cifar):
    
    num_img = pixel_data_cifar.shape[0]
    pixel_data_cifar = np.reshape(pixel_data_cifar, (num_img, 1024, 3))
    hue = cv2.cvtColor(pixel_data_cifar, cv2.COLOR_RGB2HSV)[:,:,0]

    Hue_split = np.split(hue, num_img, axis=0)

    hist = []

    for img in range(0, num_img):
        hist_row, _ = np.histogram(Hue_split[img], bins=180, range=(0, 179), density=True)
        hist = np.append(hist, hist_row)

    hist = np.reshape(hist, (num_img, 180))
    
    return hist

def classification_hog(pixel_data):
    
    pixels_per_cell = (8, 8)    # --> 16 cells
    cells_per_block = (4, 4)    # --> 1 block
    orientations = 10
    num_images = pixel_data.shape[0]
    len_hog = int(orientations * 1024/(pixels_per_cell[0]*pixels_per_cell[1]))
    pixel_data = pixel_data.reshape(num_images, 3, 32, 32).transpose(0,2,3,1).astype("float")
    hog_list = []
    for img in range(0, num_images):
        hog_feats = ski.feature.hog(pixel_data[img, :], orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, block_norm='L1', visualize=False, feature_vector=True, channel_axis=2)
        hog_list = np.append(hog_list, hog_feats)
    
    hog_list = np.reshape(hog_list, (num_images, len_hog))
    
    return hog_list





Unpickle CIFAR

In [3]:
num_train = 10000
num_val = 2000
num_test = 1000

if num_train > 50000:
    num_train = 48000
    num_val = 2000

if num_train + num_val > 50000:
    num_val = 50000 - num_train

if num_test > 10000:
    num_test = 10000


data_batch_1 = unpickle(R'sheet3\CIFAR\data_batch_1.bin')   
data_batch_2 = unpickle(R'sheet3\CIFAR\data_batch_2.bin')   
data_batch_3 = unpickle(R'sheet3\CIFAR\data_batch_3.bin')   
data_batch_4 = unpickle(R'sheet3\CIFAR\data_batch_4.bin')
data_batch_5 = unpickle(R'sheet3\CIFAR\data_batch_5.bin')
data_batch_test = unpickle(R'sheet3\CIFAR\test_batch.bin')

pixel_data_batch_1 = np.asarray(data_batch_1[b'data'])
pixel_data_batch_2 = np.asarray(data_batch_2[b'data'])
pixel_data_batch_3 = np.asarray(data_batch_3[b'data'])
pixel_data_batch_4 = np.asarray(data_batch_4[b'data'])
pixel_data_batch_5 = np.asarray(data_batch_5[b'data'])


pixel_data = np.concatenate((pixel_data_batch_1, pixel_data_batch_2, pixel_data_batch_3, pixel_data_batch_4, pixel_data_batch_5), axis=0)


labels_data_batch_1 = np.asarray(data_batch_1[b'labels'])   
labels_data_batch_2 = np.asarray(data_batch_2[b'labels'])   
labels_data_batch_3 = np.asarray(data_batch_3[b'labels'])   
labels_data_batch_4 = np.asarray(data_batch_4[b'labels'])  
labels_data_batch_5 = np.asarray(data_batch_5[b'labels'])  


labels_data = np.concatenate((labels_data_batch_1, labels_data_batch_2, labels_data_batch_3, labels_data_batch_4, labels_data_batch_5))


pixel_data_test = np.asarray(data_batch_test[b'data'])[0:num_test,:]
labels_data_test = np.asarray(data_batch_test[b'labels']) [0:num_test]

pixel_data_train = pixel_data[0:num_train,:]
labels_data_train = labels_data[0:num_train]

pixel_data_val = pixel_data[50001-num_val:-1,:]
labels_data_val = labels_data[50001-num_val+1:-1]

Convert to Hog

In [7]:
    
pixels_per_cell = (8, 8)    # --> 16 cells
cells_per_block = (4, 4)    # --> 1 block
orientations = 10
num_images = pixel_data_train.shape[0]
len_hog = int(orientations * 1024/(pixels_per_cell[0]*pixels_per_cell[1]))
pixel_data_hog = pixel_data_train.reshape(num_images, 3, 32, 32).transpose(0,2,3,1).astype("float")
hog_list = []
for img in range(0, num_images):
    hog_feats = ski.feature.hog(pixel_data_hog[img, :], orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, block_norm='L1', visualize=False, feature_vector=True, channel_axis=2)
    hog_list = np.append(hog_list, hog_feats)

hog_list = np.reshape(hog_list, (num_images, len_hog))

mean = np.mean(hog_list, dtype=np.float64)
hog_list -= mean

max_hist = np.max(hog_list)
min_hist = np.min(hog_list)
std = np.std(hog_list)

'''if max_hist > np.abs(min_hist):
    scale = max_hist
else:
    scale = np.abs(min_hist)'''

hog_norm = hog_list/std



In [6]:
print(np.max(hog_norm))
print(np.min(hog_norm))
print(np.mean(hog_norm))

1.0
-0.02363848390831497
-4.62165028469741e-18
